In [ ]:
!pip install streamlit tensorflow pillow firebase-admin

In [ ]:
import streamlit as st
import numpy as np
from tensorflow import keras
from PIL import Image
import firebase_admin
from firebase_admin import credentials, firestore
import datetime

In [ ]:
if not firebase_admin._apps:
    cred = credentials.Certificate("firebase_key.json")  # tu clave aquí
    firebase_admin.initialize_app(cred)
db = firestore.client()

In [ ]:
@st.cache_resource
def load_model():
    return keras.models.load_model("model.h5")

model = load_model()

In [ ]:
def preprocess_image(img):
    img = img.convert("L").resize((28, 28))
    img_array = np.array(img)
    if img_array.mean() > 127:
        img_array = 255 - img_array
    img_array = img_array.reshape(1, 784).astype("float32") / 255.0
    return img_array


In [ ]:
st.title("🔢 Clasificador de Dígitos con IA + Firebase")
st.write("Sube una imagen con un número escrito a mano (0-9).")

uploaded_file = st.file_uploader("Sube una imagen", type=["png", "jpg", "jpeg"])

if uploaded_file:
    img = Image.open(uploaded_file)
    st.image(img, caption="Imagen subida", use_column_width=True)

    x_img = preprocess_image(img)
    prediction = model.predict(x_img)
    digit = int(np.argmax(prediction))

    st.success(f"Predicción de la IA: **{digit}**")

    doc_ref = db.collection("predicciones").document()
    doc_ref.set({
        "prediccion": digit,
        "probabilidades": prediction.tolist(),
        "fecha": datetime.datetime.now().isoformat()
    })

    st.info("✅ Resultado guardado en Firebase")
